In [1]:
%load_ext autoreload
%autoreload 2


import pandas as pd
from datetime import timedelta
from pathlib import Path
import os
import sys
from utils.utils import get_git_root


In [2]:
project_root = get_git_root()
data_folder = "data"

In [3]:
num_prior_days = 10

In [4]:
df_dem = pd.read_csv(
    Path(project_root, data_folder, "Demographics/demographics.csv")
)
df_survey = pd.read_csv(
    Path(project_root, data_folder, "all_student_ema_data_cleaned.csv")
)
df_sens = pd.read_csv(Path(project_root, data_folder, "Sensing/sensing.csv"))

In [5]:
df_survey["day"] = pd.to_datetime(df_survey["day"], format="%Y%m%d")
df_sens["day"] = pd.to_datetime(df_sens["day"], format="%Y%m%d")
df_survey = df_survey.drop(columns=["Unnamed: 0"])

In [6]:
os = df_sens.loc[df_sens.groupby("uid")["day"].idxmin(), ["uid", "is_ios"]]

In [7]:
def calculate_cohort_year(date):
    if date.month >= 9:
        return date.year
    else:
        return date.year - 1


cohort_year = df_survey.groupby("uid")["day"].min().reset_index()
cohort_year.columns = ["uid", "first_day"]
cohort_year["cohort_year"] = cohort_year["first_day"].apply(
    calculate_cohort_year
)
cohort_year = cohort_year.drop(columns=["first_day"])

In [8]:
os.head()

,uid,is_ios
25478,003df5deff30e1e5a07b5d063fe85c3f,1
182233,0107c61e54459068bb83f6be2058d65d,1
17764,01fb41df0f6c2f69d65db5a38c600b4c,1
57827,031cf9537e5da78c5a69a10cba088c94,1
212421,03a0ce5623bfeb8aa3113605f7682215,1


In [9]:
cohort_year.head()

,uid,cohort_year
0,003df5deff30e1e5a07b5d063fe85c3f,2017
1,0107c61e54459068bb83f6be2058d65d,2018
2,01fb41df0f6c2f69d65db5a38c600b4c,2017
3,031cf9537e5da78c5a69a10cba088c94,2017
4,03a0ce5623bfeb8aa3113605f7682215,2018


In [10]:
df_sens["loc_study_product"] = df_sens.loc_study_still * df_sens.loc_study_dur

In [11]:
cols = [
    "uid",
    "day",
    # physical
    "act_on_foot_ep_0",
    "act_running_ep_0",
    "act_walking_ep_0",
    "loc_study_still",
    "loc_study_dur",
    "loc_study_product",
    "loc_home_dur",
    "loc_workout_dur",
    # social
    "act_in_vehicle_ep_0",
    "loc_dist_ep_0",
    "loc_social_dur",
    "loc_food_dur",
    "loc_visit_num_ep_0",
    "loc_social_unlock_duration",
    "loc_food_unlock_duration",
    "loc_social_unlock_num",
    "loc_food_unlock_num",
    "loc_social_still",
    "loc_food_still",
]
df_sens = df_sens[cols].copy()

In [12]:
%%time
merged_list = []

for i, uid in enumerate(df_dem['uid'].unique()):
    # print(f"{i+1} / {len(df_dem['uid'].unique())}")
    df_survey_uid = df_survey[df_survey['uid'] == uid]
    df_sens_uid = df_sens[df_sens['uid'] == uid]

    df_merged = df_survey_uid.merge(df_sens_uid, how='cross', suffixes=('_survey', '_sens'))
    df_filtered = df_merged[
        (df_merged['day_sens'] >= df_merged['day_survey'] - timedelta(days=num_prior_days)) &
        (df_merged['day_sens'] <= df_merged['day_survey'] - timedelta(days=1))
    ]
    merged_list.append(df_filtered)


CPU times: user 7.98 s, sys: 3.4 ms, total: 7.98 s
Wall time: 7.98 s


In [13]:
df_merged = pd.concat(merged_list).reset_index(drop=True)

In [14]:
df_merged[["uid_survey", "uid_sens", "day_survey", "day_sens"]]

,uid_survey,uid_sens,day_survey,day_sens
0,3569e2f520db9014b4acc4227a6421c1,3569e2f520db9014b4acc4227a6421c1,2017-09-09,2017-09-08
1,3569e2f520db9014b4acc4227a6421c1,3569e2f520db9014b4acc4227a6421c1,2017-09-10,2017-09-08
2,3569e2f520db9014b4acc4227a6421c1,3569e2f520db9014b4acc4227a6421c1,2017-09-10,2017-09-09
3,3569e2f520db9014b4acc4227a6421c1,3569e2f520db9014b4acc4227a6421c1,2017-09-14,2017-09-08
4,3569e2f520db9014b4acc4227a6421c1,3569e2f520db9014b4acc4227a6421c1,2017-09-14,2017-09-09
...,...,...,...,...
337578,2c4f43b2212eee5ba69563f139911138,2c4f43b2212eee5ba69563f139911138,2020-12-24,2020-12-19
337579,2c4f43b2212eee5ba69563f139911138,2c4f43b2212eee5ba69563f139911138,2020-12-24,2020-12-20
337580,2c4f43b2212eee5ba69563f139911138,2c4f43b2212eee5ba69563f139911138,2020-12-24,2020-12-21
337581,2c4f43b2212eee5ba69563f139911138,2c4f43b2212eee5ba69563f139911138,2020-12-24,2020-12-22


In [15]:
df_merged["diff_days"] = df_merged.day_survey - df_merged.day_sens

In [16]:
df_merged.diff_days.value_counts()

diff_days
1 days     34520
2 days     34181
3 days     34007
4 days     33888
5 days     33788
6 days     33656
7 days     33542
8 days     33425
9 days     33347
10 days    33229
Name: count, dtype: int64

In [17]:
df_grouped = df_merged.groupby(["uid_survey", "day_survey"])[cols[2:]].mean()

In [18]:
df_grouped = df_grouped.reset_index()

In [19]:
df_grouped

,uid_survey,day_survey,act_on_foot_ep_0,act_running_ep_0,act_walking_ep_0,loc_study_still,loc_study_dur,loc_study_product,loc_home_dur,loc_workout_dur,...,loc_dist_ep_0,loc_social_dur,loc_food_dur,loc_visit_num_ep_0,loc_social_unlock_duration,loc_food_unlock_duration,loc_social_unlock_num,loc_food_unlock_num,loc_social_still,loc_food_still
0,003df5deff30e1e5a07b5d063fe85c3f,2017-10-06,NaN,28.6,8367.6,54.965003,4.973667,341.745833,7.850833,0.000000,...,5314.150918,0.100056,1.192722,5.75,0.001666,12.981414,1.998890,8.104227,51.271516,39.847681
1,003df5deff30e1e5a07b5d063fe85c3f,2017-10-13,NaN,107.1,12074.9,54.090355,5.522111,335.040741,13.417500,0.099861,...,4846.948206,0.294833,1.323167,5.40,2.103271,5.354113,2.023756,6.468923,52.250667,43.290503
2,003df5deff30e1e5a07b5d063fe85c3f,2017-10-14,NaN,105.4,11648.5,54.435750,5.696083,349.405556,13.092806,0.099861,...,4713.753968,0.294833,1.373194,5.40,2.103271,4.793226,2.023756,6.659753,52.250667,43.072138
3,003df5deff30e1e5a07b5d063fe85c3f,2017-10-16,NaN,97.7,13301.6,54.632991,4.835722,334.966667,11.532833,0.099861,...,43788.508496,0.244806,1.060222,5.10,3.154074,5.145072,2.036190,5.613500,52.740242,47.374084
4,003df5deff30e1e5a07b5d063fe85c3f,2017-10-23,NaN,19.3,10984.2,55.205721,5.594250,342.609259,11.239111,0.000000,...,43090.361687,0.239306,0.689278,4.80,13.348328,11.027101,8.905356,9.161989,50.909477,43.179100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34697,ffc4b142e017c162ed4db7b05414fc4b,2021-12-18,859.9,0.0,0.0,NaN,0.000000,NaN,16.511264,0.000000,...,8560.195186,0.000000,0.000000,1.80,NaN,NaN,NaN,NaN,NaN,NaN
34698,ffc4b142e017c162ed4db7b05414fc4b,2021-12-31,447.5,0.0,0.0,NaN,0.000000,NaN,16.102055,0.000000,...,13369.742485,0.000000,0.000000,1.50,NaN,NaN,NaN,NaN,NaN,NaN
34699,ffc4b142e017c162ed4db7b05414fc4b,2022-01-27,3035.2,0.0,0.0,55.165548,20.181582,1113.827508,17.072267,0.000000,...,3000.404831,0.479104,1.472438,3.80,28.544553,25.395577,5.667650,6.578229,52.440481,47.555038
34700,ffc4b142e017c162ed4db7b05414fc4b,2022-01-30,3160.6,0.0,0.0,56.156938,20.019582,1124.678674,17.705470,0.000000,...,3131.680285,0.462189,1.163869,3.60,28.966558,29.362945,7.668096,9.844660,50.523585,45.338678


In [20]:
df = df_dem.merge(cohort_year, on="uid", how="outer")

In [21]:
df_dem

,uid,gender,race
0,3569e2f520db9014b4acc4227a6421c1,both,white
1,ac70fe1f8115ac361f2023269c011c3e,M,asian
2,3bb377ba0acb7d8916010184df36aa57,F,white
3,fa394f6d3d077bd5568fc3bc01580806,F,white
4,84120765740b5395aa49a2feb12fbb43,M,asian
...,...,...,...
211,fe8ddda4ae8c71f7054ca024b82f5c98,F,white
212,f5529fbae87a8d170937d3e39d5a63cc,F,white
213,a52b5e80b4c7a8e05f8cc0a16ae4ea9f,F,black
214,6b0083d00297f9c03e00b2cde889b666,M,asian


In [22]:
df.head()

,uid,gender,race,cohort_year
0,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017
1,0107c61e54459068bb83f6be2058d65d,F,white,2018
2,01fb41df0f6c2f69d65db5a38c600b4c,M,white,2017
3,031cf9537e5da78c5a69a10cba088c94,F,white,2017
4,03a0ce5623bfeb8aa3113605f7682215,F,more than one,2018


In [23]:
df = df.merge(os, on="uid", how="outer")

In [24]:
df.head()

,uid,gender,race,cohort_year,is_ios
0,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1
1,0107c61e54459068bb83f6be2058d65d,F,white,2018.0,1
2,01fb41df0f6c2f69d65db5a38c600b4c,M,white,2017.0,1
3,031cf9537e5da78c5a69a10cba088c94,F,white,2017.0,1
4,03a0ce5623bfeb8aa3113605f7682215,F,more than one,2018.0,1


In [25]:
df = df.merge(df_grouped, left_on="uid", right_on="uid_survey", how="outer")

In [26]:
df.head()

,uid,gender,race,cohort_year,is_ios,uid_survey,day_survey,act_on_foot_ep_0,act_running_ep_0,act_walking_ep_0,...,loc_dist_ep_0,loc_social_dur,loc_food_dur,loc_visit_num_ep_0,loc_social_unlock_duration,loc_food_unlock_duration,loc_social_unlock_num,loc_food_unlock_num,loc_social_still,loc_food_still
0,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,003df5deff30e1e5a07b5d063fe85c3f,2017-10-06,NaN,28.6,8367.6,...,5314.150918,0.100056,1.192722,5.75,0.001666,12.981414,1.998890,8.104227,51.271516,39.847681
1,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,003df5deff30e1e5a07b5d063fe85c3f,2017-10-13,NaN,107.1,12074.9,...,4846.948206,0.294833,1.323167,5.40,2.103271,5.354113,2.023756,6.468923,52.250667,43.290503
2,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,003df5deff30e1e5a07b5d063fe85c3f,2017-10-14,NaN,105.4,11648.5,...,4713.753968,0.294833,1.373194,5.40,2.103271,4.793226,2.023756,6.659753,52.250667,43.072138
3,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,003df5deff30e1e5a07b5d063fe85c3f,2017-10-16,NaN,97.7,13301.6,...,43788.508496,0.244806,1.060222,5.10,3.154074,5.145072,2.036190,5.613500,52.740242,47.374084
4,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,003df5deff30e1e5a07b5d063fe85c3f,2017-10-23,NaN,19.3,10984.2,...,43090.361687,0.239306,0.689278,4.80,13.348328,11.027101,8.905356,9.161989,50.909477,43.179100


In [27]:
df = df.drop(columns=["uid_survey"])

In [28]:
df.head()

,uid,gender,race,cohort_year,is_ios,day_survey,act_on_foot_ep_0,act_running_ep_0,act_walking_ep_0,loc_study_still,...,loc_dist_ep_0,loc_social_dur,loc_food_dur,loc_visit_num_ep_0,loc_social_unlock_duration,loc_food_unlock_duration,loc_social_unlock_num,loc_food_unlock_num,loc_social_still,loc_food_still
0,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-06,NaN,28.6,8367.6,54.965003,...,5314.150918,0.100056,1.192722,5.75,0.001666,12.981414,1.998890,8.104227,51.271516,39.847681
1,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-13,NaN,107.1,12074.9,54.090355,...,4846.948206,0.294833,1.323167,5.40,2.103271,5.354113,2.023756,6.468923,52.250667,43.290503
2,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-14,NaN,105.4,11648.5,54.435750,...,4713.753968,0.294833,1.373194,5.40,2.103271,4.793226,2.023756,6.659753,52.250667,43.072138
3,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-16,NaN,97.7,13301.6,54.632991,...,43788.508496,0.244806,1.060222,5.10,3.154074,5.145072,2.036190,5.613500,52.740242,47.374084
4,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-23,NaN,19.3,10984.2,55.205721,...,43090.361687,0.239306,0.689278,4.80,13.348328,11.027101,8.905356,9.161989,50.909477,43.179100


In [29]:
df["act_running_ep_0_plus_act_walking_ep_0"] = (
    df.act_running_ep_0 + df.act_walking_ep_0
)

In [30]:
df["loc_social_dur_plus_loc_food_dur"] = df.loc_social_dur + df.loc_food_dur

In [31]:
df["loc_social_unlock_duration_plus_loc_food_unlock_duration"] = (
    df.loc_social_unlock_duration + df.loc_food_unlock_duration
)

In [32]:
df["loc_social_unlock_num_plus_loc_food_unlock_num"] = (
    df.loc_social_unlock_num + df.loc_food_unlock_num
)

In [33]:
df["loc_social_still_plus_loc_food_still"] = (
    df.loc_social_still + df.loc_food_still
)

In [34]:
df.head(1).T

,0
uid,003df5deff30e1e5a07b5d063fe85c3f
gender,F
race,asian
cohort_year,2017.0
is_ios,1
day_survey,2017-10-06 00:00:00
act_on_foot_ep_0,NaN
act_running_ep_0,28.6
act_walking_ep_0,8367.6
loc_study_still,54.965003


# check


In [35]:
df_sens = pd.read_csv(Path(project_root, data_folder, "Sensing/sensing.csv"))
df_sens["day"] = pd.to_datetime(df_sens["day"], format="%Y%m%d")

In [36]:
os

,uid,is_ios
25478,003df5deff30e1e5a07b5d063fe85c3f,1
182233,0107c61e54459068bb83f6be2058d65d,1
17764,01fb41df0f6c2f69d65db5a38c600b4c,1
57827,031cf9537e5da78c5a69a10cba088c94,1
212421,03a0ce5623bfeb8aa3113605f7682215,1
...,...,...
120060,fbcc8cd8254960ed44ffdd3390a2f6a0,1
135436,fc26652471ae91bdbbc11a8be89c362c,0
136468,fc73375dda5e5460f7088c78654a945e,1
213727,fe8ddda4ae8c71f7054ca024b82f5c98,1


In [37]:
uid = "fe8ddda4ae8c71f7054ca024b82f5c98"

df_sens.loc[df_sens.uid == uid].sort_values(
    by="day", ascending=True
).is_ios.head(1)

213727    1
Name: is_ios, dtype: int64

In [38]:
df

,uid,gender,race,cohort_year,is_ios,day_survey,act_on_foot_ep_0,act_running_ep_0,act_walking_ep_0,loc_study_still,...,loc_food_unlock_duration,loc_social_unlock_num,loc_food_unlock_num,loc_social_still,loc_food_still,act_running_ep_0_plus_act_walking_ep_0,loc_social_dur_plus_loc_food_dur,loc_social_unlock_duration_plus_loc_food_unlock_duration,loc_social_unlock_num_plus_loc_food_unlock_num,loc_social_still_plus_loc_food_still
0,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-06,NaN,28.6,8367.6,54.965003,...,12.981414,1.998890,8.104227,51.271516,39.847681,8396.2,1.292778,12.983080,10.103116,91.119197
1,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-13,NaN,107.1,12074.9,54.090355,...,5.354113,2.023756,6.468923,52.250667,43.290503,12182.0,1.618000,7.457384,8.492680,95.541169
2,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-14,NaN,105.4,11648.5,54.435750,...,4.793226,2.023756,6.659753,52.250667,43.072138,11753.9,1.668028,6.896497,8.683509,95.322805
3,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-16,NaN,97.7,13301.6,54.632991,...,5.145072,2.036190,5.613500,52.740242,47.374084,13399.3,1.305028,8.299145,7.649689,100.114327
4,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-23,NaN,19.3,10984.2,55.205721,...,11.027101,8.905356,9.161989,50.909477,43.179100,11003.5,0.928583,24.375429,18.067345,94.088577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34701,ffc4b142e017c162ed4db7b05414fc4b,F,black,2018.0,0,2021-12-18,859.9,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN
34702,ffc4b142e017c162ed4db7b05414fc4b,F,black,2018.0,0,2021-12-31,447.5,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN
34703,ffc4b142e017c162ed4db7b05414fc4b,F,black,2018.0,0,2022-01-27,3035.2,0.0,0.0,55.165548,...,25.395577,5.667650,6.578229,52.440481,47.555038,0.0,1.951542,53.940130,12.245879,99.995518
34704,ffc4b142e017c162ed4db7b05414fc4b,F,black,2018.0,0,2022-01-30,3160.6,0.0,0.0,56.156938,...,29.362945,7.668096,9.844660,50.523585,45.338678,0.0,1.626057,58.329504,17.512757,95.862263


In [39]:
df.to_csv("features.csv", index=False)

In [40]:
check_df = pd.read_csv("features.csv")
check_df

,uid,gender,race,cohort_year,is_ios,day_survey,act_on_foot_ep_0,act_running_ep_0,act_walking_ep_0,loc_study_still,...,loc_food_unlock_duration,loc_social_unlock_num,loc_food_unlock_num,loc_social_still,loc_food_still,act_running_ep_0_plus_act_walking_ep_0,loc_social_dur_plus_loc_food_dur,loc_social_unlock_duration_plus_loc_food_unlock_duration,loc_social_unlock_num_plus_loc_food_unlock_num,loc_social_still_plus_loc_food_still
0,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-06,NaN,28.6,8367.6,54.965003,...,12.981414,1.998890,8.104227,51.271516,39.847681,8396.2,1.292778,12.983080,10.103116,91.119197
1,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-13,NaN,107.1,12074.9,54.090355,...,5.354113,2.023756,6.468923,52.250667,43.290503,12182.0,1.618000,7.457384,8.492680,95.541169
2,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-14,NaN,105.4,11648.5,54.435750,...,4.793226,2.023756,6.659753,52.250667,43.072138,11753.9,1.668028,6.896497,8.683509,95.322805
3,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-16,NaN,97.7,13301.6,54.632991,...,5.145072,2.036190,5.613500,52.740242,47.374084,13399.3,1.305028,8.299145,7.649689,100.114327
4,003df5deff30e1e5a07b5d063fe85c3f,F,asian,2017.0,1,2017-10-23,NaN,19.3,10984.2,55.205721,...,11.027101,8.905356,9.161989,50.909477,43.179100,11003.5,0.928583,24.375429,18.067345,94.088577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34701,ffc4b142e017c162ed4db7b05414fc4b,F,black,2018.0,0,2021-12-18,859.9,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN
34702,ffc4b142e017c162ed4db7b05414fc4b,F,black,2018.0,0,2021-12-31,447.5,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN
34703,ffc4b142e017c162ed4db7b05414fc4b,F,black,2018.0,0,2022-01-27,3035.2,0.0,0.0,55.165548,...,25.395577,5.667650,6.578229,52.440481,47.555038,0.0,1.951542,53.940130,12.245879,99.995518
34704,ffc4b142e017c162ed4db7b05414fc4b,F,black,2018.0,0,2022-01-30,3160.6,0.0,0.0,56.156938,...,29.362945,7.668096,9.844660,50.523585,45.338678,0.0,1.626057,58.329504,17.512757,95.862263


In [41]:
check_df.columns

Index(['uid', 'gender', 'race', 'cohort_year', 'is_ios', 'day_survey',
       'act_on_foot_ep_0', 'act_running_ep_0', 'act_walking_ep_0',
       'loc_study_still', 'loc_study_dur', 'loc_study_product', 'loc_home_dur',
       'loc_workout_dur', 'act_in_vehicle_ep_0', 'loc_dist_ep_0',
       'loc_social_dur', 'loc_food_dur', 'loc_visit_num_ep_0',
       'loc_social_unlock_duration', 'loc_food_unlock_duration',
       'loc_social_unlock_num', 'loc_food_unlock_num', 'loc_social_still',
       'loc_food_still', 'act_running_ep_0_plus_act_walking_ep_0',
       'loc_social_dur_plus_loc_food_dur',
       'loc_social_unlock_duration_plus_loc_food_unlock_duration',
       'loc_social_unlock_num_plus_loc_food_unlock_num',
       'loc_social_still_plus_loc_food_still'],
      dtype='object')